In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
def rmse(y_true,y_preds):
    return np.sqrt(mean_squared_error(y_true,y_preds))

In [3]:
df = pd.read_excel('data/Tax Simulation for Data science.xlsx')

In [4]:
df.head()

,Unnamed: 0,Basic,Housing,Transport,Other Taxable Allowances,Other Non Taxable Allowances,Gross,Taxable Gross,Pension,NHF,Personal\nrelief,Standard Relief,Total Relief,Net Taxable,Computed Tax,True Tax,NetPay adjusted for Tax Only,NetPay adjusted for Tax+Pension+NHF
0,NaN,10000,NaN,NaN,NaN,NaN,10000,10000,800,250,16666.66,2000,19716.66,-9716.66,-680.1662,100.0,9900.0,8850.0
1,NaN,11000,NaN,NaN,NaN,NaN,11000,11000,880,275,16666.66,2200,20021.66,-9021.66,-631.5162,110.0,10890.0,9735.0
2,NaN,12000,NaN,NaN,NaN,NaN,12000,12000,960,300,16666.66,2400,20326.66,-8326.66,-582.8662,120.0,11880.0,10620.0
3,NaN,13000,NaN,NaN,NaN,NaN,13000,13000,1040,325,16666.66,2600,20631.66,-7631.66,-534.2162,130.0,12870.0,11505.0
4,NaN,14000,NaN,NaN,NaN,NaN,14000,14000,1120,350,16666.66,2800,20936.66,-6936.66,-485.5662,140.0,13860.0,12390.0


In [5]:
df.columns

Index(['Unnamed: 0', 'Basic', 'Housing', 'Transport',
       'Other Taxable Allowances', 'Other Non Taxable Allowances', 'Gross',
       'Taxable Gross', 'Pension', 'NHF', 'Personal\nrelief',
       'Standard Relief', 'Total Relief', 'Net Taxable', 'Computed Tax',
       'True Tax', 'NetPay adjusted for Tax Only',
       'NetPay adjusted for Tax+Pension+NHF'],
      dtype='object')

In [6]:
df.drop(['Unnamed: 0', 'Housing', 'Transport',
       'Other Taxable Allowances', 'Other Non Taxable Allowances', 'Gross',
       'Taxable Gross', 'Pension', 'NHF', 'Personal\nrelief',
       'Standard Relief', 'Total Relief', 'Net Taxable', 'Computed Tax',
       'True Tax', 'NetPay adjusted for Tax Only'], axis=1, inplace=True)

In [7]:
df.drop(df.index[0:20],axis=0, inplace=True)

In [8]:
def extract_features(df,col):
    df['Pension'] = 0.08 * df[col]
    df['NHF'] = 0.025 * df[col]
    df['Standard_relief'] = 0.2 * df[col]
    df['Personal_relief'] = df[col].apply(lambda x: (200000/12) if x < (20000000/12) else 0.01* x)
    df['Total_relief'] = df['Pension'] + df['NHF'] + df['Personal_relief'] + df['Standard_relief']
    df['Net_taxable'] = df[col] - df['Total_relief']
    return df

In [9]:
df.reset_index(inplace=True,drop=True)

In [10]:
df = extract_features(df,'Basic')

In [11]:
df

,Basic,NetPay adjusted for Tax+Pension+NHF,Pension,NHF,Standard_relief,Personal_relief,Total_relief,Net_taxable
0,30000,2.655000e+04,2400.0,750.0,6000.0,16666.666667,2.581667e+04,4.183333e+03
1,31000,2.740352e+04,2480.0,775.0,6200.0,16666.666667,2.612167e+04,4.878333e+03
2,32000,2.824987e+04,2560.0,800.0,6400.0,16666.666667,2.642667e+04,5.573333e+03
3,33000,2.909622e+04,2640.0,825.0,6600.0,16666.666667,2.673167e+04,6.268333e+03
4,34000,2.994257e+04,2720.0,850.0,6800.0,16666.666667,2.703667e+04,6.963333e+03
...,...,...,...,...,...,...,...,...
20451,20481000,1.498075e+07,1638480.0,512025.0,4096200.0,204810.000000,6.451515e+06,1.402948e+07
20452,20482000,1.498148e+07,1638560.0,512050.0,4096400.0,204820.000000,6.451830e+06,1.403017e+07
20453,20483000,1.498221e+07,1638640.0,512075.0,4096600.0,204830.000000,6.452145e+06,1.403086e+07
20454,20484000,1.498294e+07,1638720.0,512100.0,4096800.0,204840.000000,6.452460e+06,1.403154e+07


In [12]:
X = df.drop('NetPay adjusted for Tax+Pension+NHF',1)
y = df['NetPay adjusted for Tax+Pension+NHF']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [15]:
X_test.head()

,Basic,Pension,NHF,Standard_relief,Personal_relief,Total_relief,Net_taxable
326,356000,28480.0,8900.0,71200.0,16666.666667,1.252467e+05,2.307533e+05
3367,3397000,271760.0,84925.0,679400.0,33970.000000,1.070055e+06,2.326945e+06
6761,6791000,543280.0,169775.0,1358200.0,67910.000000,2.139165e+06,4.651835e+06
2627,2657000,212560.0,66425.0,531400.0,26570.000000,8.369550e+05,1.820045e+06
9398,9428000,754240.0,235700.0,1885600.0,94280.000000,2.969820e+06,6.458180e+06


In [16]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [18]:
model_pipeline = Pipeline([('polynomial_features',PolynomialFeatures(degree=2)),
                           ('model',LinearRegression())])

In [19]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('polynomial_features', PolynomialFeatures()),
                ('model', LinearRegression())])

In [20]:
y_preds = model_pipeline.predict(X_test)

In [21]:
rmse(y_test,y_preds)

513.3748932341953

In [ ]:
import pickle

In [ ]:
pickle.dump(model_pipeline,open('netpay_regressor.pkl','wb'))
pickle.dump(scaler,open('netpay_scaler.pkl','wb'))